In [6]:
import sys
sys.path.append("../methods/")

from tqdm import trange
from paus import *
from utils import *
from mirror_prox import *
from saddle_sliding import *

In [7]:
k = 5 # grid size
n = k * k # amount of houses
distance_matrix = get_distance_matrix(make_house_grid(k))
theta = 2 / distance_matrix.max()

matrices_array = [get_utility_matrix(distance_matrix) for _ in range(10000)]
mean_matrix = np.mean(matrices_array, axis=0)

machine_1 = matrices_array[:2000]
machine_2 = matrices_array[2000:4000]
machine_3 = matrices_array[4000:6000]
machine_4 = matrices_array[6000:8000]
machine_5 = matrices_array[8000:]

A_1 = np.mean(machine_1, axis=0)
A_2 = np.mean(machine_2, axis=0)
A_3 = np.mean(machine_3, axis=0)
A_4 = np.mean(machine_4, axis=0)
A_5 = np.mean(machine_5, axis=0)

In [8]:
delta = np.linalg.norm(get_hess(mean_matrix) - get_hess(A_1), ord=1)
Lipschitz = np.max(mean_matrix)
gamma_sim = 1 / delta

print("delta =  ", delta, "\n")
print("Lipschitz =  ", Lipschitz, "\n")
print("gamma_sim =  ", gamma_sim)

delta =   0.3041361980867748 

Lipschitz =   0.8679504426871004 

gamma_sim =   3.288000594111078


or you can try another hyperparameters

In [ ]:
delta = np.max(A_1)
Lipschitz = np.max(mean_matrix)
gamma_sim = 1 / delta

print("delta =  ", delta, "\n")
print("Lipschitz =  ", Lipschitz, "\n")
print("gamma_sim =  ", gamma_sim)

In [9]:
iter_grad_calls = 4

In [10]:
def run_one_log(c_list: list, name_of_method: str):
    first_iteration = True
    for c in c_list:

        if name_of_method == "MirrorProx":
            ans, error = MirrorProx(grad_x, grad_y, prox_simplex, gamma_MP, c, n, Lipschitz, mean_matrix)
            x = np.array(range(len(error))) * iter_grad_calls

            if first_iteration:
                plt.loglog(x, error, label = r'$\gamma$ (theoretical) = {}'.format(round(c / Lipschitz, 2) if c / Lipschitz < 1.0 else round(c / Lipschitz)))
                first_iteration = False

            else:
                plt.loglog(x, error, label = r'$\gamma$ = {}'.format(round(c / Lipschitz, 2) if c / Lipschitz < 1.0 else round(c / Lipschitz)))

            continue
        
        elif name_of_method == "PAUS":
            ans_breg, error_breg = PAUS_method(grad_x, grad_y, prox_simplex, gamma_MP, c, n, Lipschitz, gamma_sim, A_1, mean_matrix)
            x = np.array(range(len(error_breg))) * iter_grad_calls

            if first_iteration:
                plt.loglog(x, error_breg, label = r'$\gamma$ (theoretical) = {}'.format(round(c / delta, 2) if c / delta < 1.0 else round(c / delta)))
                first_iteration = False

            else:
                plt.loglog(x, error_breg, label = r'$\gamma$ = {}'.format(round(c / delta, 2) if c / delta < 1.0 else round(c / delta)))

            continue

        elif name_of_method == "Euclidean":
            ans, error_sad = Euclidean_method(grad_x, grad_y, prox_simplex, gamma_MP, c, n, Lipschitz, gamma_sim, A_1, mean_matrix)
            x = np.array(range(len(error_sad))) * iter_grad_calls

            if first_iteration:
                plt.loglog(x, error_sad, label = r'$\gamma$ (theoretical) = {}'.format(round(c / delta, 2) if c / delta < 1.0 else round(c / delta)))
                first_iteration = False

            else:
                plt.loglog(x, error_sad, label = r'$\gamma$ = {}'.format(round(c / delta, 2) if c / delta < 1.0 else round(c / delta)))

            continue

    plt.title(name_of_method, fontsize=20)
    plt.xlabel("communications", fontsize=20)
    plt.ylabel("duality gap", fontsize=20)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
run_one_log([1, 50, 100, 200, 400], "MirrorProx")

In [ ]:
run_one_log([1, 50, 100, 200, 400], "PAUS")

In [ ]:
run_one_log([1, 50, 100, 200, 400], "Euclidean")